<a href="https://colab.research.google.com/github/Btere/btereml/blob/main/Arrythmia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import numpy.typing as npt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
import logging
from typing import List
import itertools
import logging
import torch
import matplotlib.colors as colors
import numpy as np
import numpy.typing as npt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score
from typing import NewType, Tuple, TypedDict, Dict, List
%matplotlib inline



# New section

Arrhythmia detection: This is irregular hearbeat.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

logger = logging.getLogger(__name__)

logging.basicConfig(level=logging.INFO)

In [ ]:
DATASET_PATH = Path("/content/drive/MyDrive/ECG_dataset/dataset/mit-bih_arrhythmia_database.csv")

load_dataset = pd.read_csv(DATASET_PATH)
load_dataset


In [ ]:
load_dataset.size

In [ ]:
load_dataset.shape

In [ ]:
x = torch.zeros(3,3)

In [ ]:
print("shape of dataset",load_dataset.shape)
print("size of dataset", load_dataset.size)

In [ ]:
load_dataset.nunique()

In [ ]:
load_dataset["type"].value_counts()

Transform multi-class labels into binary-class (arrhythmia and normal)

In [ ]:
load_dataset['target'] = load_dataset.type.replace(['VEB', 'SVEB', 'F', 'Q'], 'arrhythmia').replace(['N'], 'normal')
load_dataset.target.unique()

In [ ]:
load_dataset[load_dataset.target=='arrhythmia']

In [ ]:
#drop the record column because its the id for the patient and the type because we have the target column

new_dataframe = load_dataset.drop(columns=['record','type'])

new_dataframe.head(20000)

In [ ]:
print("info about dataset", new_dataframe.info())


In [ ]:
new_dataframe["target"].value_counts()

In [ ]:
def resample(dataset:pd.DataFrame, label: str, n_sample: int)-> pd.DataFrame:
  """Resamples the dataset to have the same amount of data.

    Args:
        dataset: The input dataset to be resampled.
        label: The column with label values
        n_sample: The desired number of samples for each class after resampling.

    Returns:
          Resampled dataset with the specified number of samples for the given class.
  """

  data = dataset[dataset['target'] == label].sample(n=n_sample)
  return data

new_data: List[pd.DataFrame] = []

for label in new_dataframe.target.unique():
  new_data.append(resample(new_dataframe,label, 10606))

new_dataset = pd.concat(new_data)
new_dataset = new_dataset.reset_index(drop=True)


In [ ]:
def plot_confusion_matrix(confusion_matrix: npt.NDArray[np.int64], class_name: List[str], normalize: bool = False, title: str = 'Confusion matrix',
                          cmap: colors.LinearSegmentedColormap = plt.cm.Blues) -> None:
    """Prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    if normalize:
        confusion_matrix = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(len(class_name), len(class_name)))
    sns.heatmap(confusion_matrix, annot=True, fmt=".2f" if normalize else "d", cmap=cmap, xticklabels=class_name, yticklabels=class_name)

    plt.title(title)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

In [ ]:
new_dataset.nunique()

Split data into training and testing set

In [ ]:
xlabel = new_dataset.iloc[:, :-1]
xlabel.shape

In [ ]:
ylabel = new_dataset[["target"]]
ylabel.value_counts()
ylabel.shape

Data visualizing

In [ ]:
numeric_cols = ylabel.select_dtypes(include=['float64', 'int64']).columns

# Filtering for Normal and Arrhythmia
normal_data = ylabel[ylabel['target'] == 'normal']
arrhythmia_data = ylabel[ylabel['target'] == 'arrhythmia']

# Plot histograms for Normal
for col in numeric_cols:
    plt.figure(figsize=(8, 6))
    plt.hist(normal_data[col], bins=30, color='blue', alpha=0.7, label='Normal')
    plt.title(f'Histogram of {col} for Normal')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot histograms for Arrhythmia
for col in numeric_cols:
    plt.figure(figsize=(8, 6))
    plt.hist(arrhythmia_data[col], bins=30, color='orange', alpha=0.7, label='Arrhythmia')
    plt.title(f'Histogram of {col} for Arrhythmia')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.show()




In [ ]:
sns.countplot(y=ylabel["target"])
plt.show()



In [ ]:
# Split data into train and test set

X_train, X_test, y_train, y_test = train_test_split(xlabel, ylabel, random_state=101)


In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Model training

model = RandomForestClassifier(random_state=101, n_estimators=150)
model.fit(X_train, y_train)

# training accuracy
print('Accuracy for the train data', model.score(X_train, y_train.values.ravel()))

In [ ]:
# model evaluation

y_pred = model.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
def _model_training(dataset:pd.DataFrame, xdata: str, ydata: str, test_size: float = 0.2, random_state: int = 101, n_estimators=150)-> Tuple[RandomForestClassifier, pd.DataFrame, pd.Series]:
  """
    Splits the dataset, trains, and evaluates the model.

    Args:
        dataset: The input dataset.
        xdata: The column with feature values.
        ydata: The column with label values.
        test_size: The proportion of the dataset to include in the test split.
        random_state: Seed for random number generation.

    Returns: None
  """
  xlabel = new_dataset.iloc[:, :-1]
  ylabel = new_dataset[['target']]

  label_encoder = LabelEncoder()
  ydata_encoded = label_encoder.fit_transform(ylabel)

  X_train, X_test, y_train, y_test = train_test_split(xlabel, ydata_encoded, test_size=test_size)
  scaler= StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  model = RandomForestClassifier(random_state=random_state, n_estimators=n_estimators)
  model.fit(X_train, y_train)

  return model, X_train, y_train


In [ ]:
def _eval_metrics(y_test: pd.Series, y_pred: pd.Series) -> Tuple[float, float,float,float]:
  """
    Evaluates a trained RandomForestClassifier and log results.

    Args:
        clf: Trained classifier.
        X_test: Features for evaluation.
        y_test: True labels for evaluation.
  """
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)

  return accuracy, f1, precision, recall

accuracy, f1, precision,recall = _eval_metrics(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(confusion_matrix, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');


In [ ]:
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear',C=10, max_iter=1000, verbose=True)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)
print("SVM Accuracy:", svm_accuracy)

In [ ]:
# DataLoader
def prepare_dataloaders(xtrain: npt.NDArray[np.float64], ytrain: npt.NDArray[np.float64],
                        xtest:npt.NDArray[np.float64],ytest:npt.NDArray[np.float64],
                        batch_size:int = 64, shuffle: bool = True) -> Tuple[DataLoader, DataLoader]:
                        """Prepare train and test dataloaders
                            Args:
                          xtrain: Data used as train input.
                          ytrain: Data used as train output.
                          xtest: Data used as test input.
                          ytest: Data used as test output.
                          batch_size: Number of samples that will be propagated through the network
                          shuffle: If true- randomly shuffles data.

                          Returns:
                          Dataloaders for train and test.
                          """
                          train_dataset = TensorDataset(torch.Tensor(xtrain), torch.Tensor(ytrain))
                          train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

                          test_dataset = TensorDataset(torch.Tensor(xtest), torch.Tensor(ytest))
                          test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

                          return train_dataloader, test_dataloader
